In [8]:
from dotenv import load_dotenv

load_dotenv()

import boto3
import os

s3 = boto3.client(
    's3',
)

bucket_name = 'kursai'

In [9]:
response = s3.list_objects_v2(Bucket=bucket_name)

for obj in response['Contents']:
    print(obj['Key'])

best_model_LinearRegression.pkl
halfmarathon_wroclaw_2023__final.csv
halfmarathon_wroclaw_2024__final.csv


In [10]:
%pip install scikit-learn

import pandas as pd
import numpy as np
import pandera as pa
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.feature_selection import SelectKBest, f_regression
import joblib
import boto3
import os

Note: you may need to restart the kernel to use updated packages.


In [11]:
#Wczytanie i połączenie danych
# Wczytujemy oba pliki CSV i łączymy je w jeden DataFrame.

df_2023 = pd.read_csv(f"s3://{bucket_name}/halfmarathon_wroclaw_2023__final.csv", sep=';')
df_2024 = pd.read_csv(f"s3://{bucket_name}/halfmarathon_wroclaw_2024__final.csv", sep=';')
df = pd.concat([df_2023, df_2024], ignore_index=True)
print('Dane połączone:', df.shape)

Dane połączone: (21957, 27)


In [12]:
# Walidacja i czyszczenie danych (pandera)
# Tworzymy schemat walidacji i czyścimy dane zgodnie z nim.
schema = pa.infer_schema(df)
df = schema.validate(df)
df = df.drop_duplicates()
df = df.dropna()
print('Dane po czyszczeniu:', df.shape)

Dane po czyszczeniu: (7009, 27)


In [13]:

# Wybieramy cechy na podstawie metryk statystycznych. Zakładamy, że przewidujemy kolumnę 'Czas'
target = 'Czas'
X = df.drop(columns=[target])
y = df[target]

# Konwersja czasu z formatu HH:MM:SS do liczby sekund
def time_to_seconds(t):
	if isinstance(t, str) and ':' in t:
		parts = t.split(':')
		if len(parts) == 3:
			h, m, s = map(int, parts)
			return h * 3600 + m * 60 + s
		elif len(parts) == 2:
			m, s = map(int, parts)
			return m * 60 + s
	return np.nan

y_numeric = y.apply(time_to_seconds)

X = pd.get_dummies(X, drop_first=True)
selector = SelectKBest(score_func=f_regression, k=min(10, X.shape[1]))
X_selected = selector.fit_transform(X, y_numeric)
selected_features = X.columns[selector.get_support()]
X = X[selected_features]
print('Wybrane cechy:', list(selected_features))


Wybrane cechy: ['Miejsce', 'Płeć Miejsce', '5 km Miejsce Open', '5 km Tempo', '10 km Miejsce Open', '10 km Tempo', '15 km Miejsce Open', '15 km Tempo', '20 km Miejsce Open', '20 km Tempo']


In [14]:
#Podział na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Trening i porównanie modeli scikit-learn
# Trenujemy kilka modeli i porównujemy ich skuteczność.
models = {
    'LinearRegression': LinearRegression(),
    'RandomForest': RandomForestRegressor(random_state=42)
    }
results = {}
for name, model in models.items():
    # Convert y_train and y_test to seconds using the same logic as y_numeric
    y_train_numeric = y_train.apply(time_to_seconds)
    y_test_numeric = y_test.apply(time_to_seconds)
    model.fit(X_train, y_train_numeric)
    y_pred = model.predict(X_test)
    results[name] = {
        'MAE': mean_absolute_error(y_test_numeric, y_pred),
        'MSE': mean_squared_error(y_test_numeric, y_pred),
        'R2': r2_score(y_test_numeric, y_pred)
    }
print('Porównanie modeli:')
for name, metrics in results.items():
    print(f"{name}: {metrics}")

# Wybór najlepszego modelu
sorted_models = sorted(
    results.items(),
    key=lambda x: (-x[1]['R2'], x[1]['MAE'], x[1]['MSE'])
)
best_model_name = sorted_models[0][0]
best_model = models[best_model_name]
print(f"\nNajlepszy model (wg R2, MAE, MSE): {best_model_name}")


Porównanie modeli:
LinearRegression: {'MAE': 22.321375437539345, 'MSE': 848.3973835513754, 'R2': 0.9994540210313967}
RandomForest: {'MAE': 17.881141226818833, 'MSE': 1629.6846419400854, 'R2': 0.99895123021687}

Najlepszy model (wg R2, MAE, MSE): LinearRegression


In [ ]:
# ...po wczytaniu i czyszczeniu danych...

# Przygotuj tylko te cechy, które możesz wyciągnąć z tekstu
def time_to_seconds(t):
    if isinstance(t, str) and ':' in t:
        parts = t.split(':')
        if len(parts) == 3:
            h, m, s = map(int, parts)
            return h * 3600 + m * 60 + s
        elif len(parts) == 2:
            m, s = map(int, parts)
            return m * 60 + s
    return np.nan

df['Czas_numeric'] = df['Czas'].apply(time_to_seconds)

# Zakładamy, że masz kolumny: 'Płeć', 'Wiek', 'Tempo_5km' (dostosuj nazwy do swoich danych!)
df['gender'] = df['Płeć'].map({'M': 0, 'K': 1})  # dostosuj mapowanie do wartości w Twojej kolumnie 'Płeć'
current_year = pd.Timestamp.now().year
df['age'] = current_year - df['Rocznik']
df['pace_5k'] = df['5 km Tempo']  # już w sekundach na km w Twoich danych

X = df[['gender', 'age', 'pace_5k']]
y = df['Czas_numeric']

X = X.dropna()
y = y.loc[X.index]

print('Używane cechy:', X.columns.tolist())

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import Pipeline

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

models = {
    'LinearRegression': LinearRegression(),
    'RandomForest': RandomForestRegressor(random_state=42)
}
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    results[name] = {
        'MAE': mean_absolute_error(y_test, y_pred),
        'MSE': mean_squared_error(y_test, y_pred),
        'R2': r2_score(y_test, y_pred)
    }
print('Porównanie modeli:')
for name, metrics in results.items():
    print(f"{name}: {metrics}")

# Wybierz najlepszy model
sorted_models = sorted(
    results.items(),
    key=lambda x: (-x[1]['R2'], x[1]['MAE'], x[1]['MSE'])
)
best_model_name = sorted_models[0][0]
best_model = models[best_model_name]
print(f"\nNajlepszy model (wg R2, MAE, MSE): {best_model_name}")

# Pipeline (tu nie potrzebujesz selektora cech)
pipeline = Pipeline([
    ('model', best_model)
])
pipeline.fit(X_train, y_train)

import joblib
model_path = "best_model_pipeline.joblib"
joblib.dump(pipeline, model_path)
print(f"Pipeline zapisany do pliku: {model_path}")

# Upload do Digital Ocean Spaces (jak wcześniej)
import boto3
session = boto3.session.Session()
client = session.client('s3')
bucket = 'kursai'
client.upload_file(model_path, bucket, model_path)
print(f"Pipeline wysłany do Digital Ocean Spaces: {bucket}/{model_path}")
print("Przykładowe wartości y (sekundy):", y.head())
print("Przykładowe wartości pace_5k:", X['pace_5k'].head())
print("Statystyki czasu:", y.describe())

Używane cechy: ['gender', 'age', 'pace_5k']
Porównanie modeli:
LinearRegression: {'MAE': 279.6483647412662, 'MSE': 142174.92719967576, 'R2': 0.9085045267480649}
RandomForest: {'MAE': 309.4213135345198, 'MSE': 178743.20461004108, 'R2': 0.8849713207632263}

Najlepszy model (wg R2, MAE, MSE): LinearRegression
Pipeline zapisany do pliku: best_model_pipeline.joblib
Pipeline wysłany do Digital Ocean Spaces: kursai/best_model_pipeline.joblib
Przykładowe wartości y (sekundy): Series([], Name: Czas_numeric, dtype: int64)
Przykładowe wartości pace_5k: Series([], Name: pace_5k, dtype: float64)
Statystyki czasu: count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: Czas_numeric, dtype: float64


In [25]:
# ...po wczytaniu i czyszczeniu danych...

def time_to_seconds(t):
    if isinstance(t, str) and ':' in t:
        parts = t.split(':')
        if len(parts) == 3:
            h, m, s = map(int, parts)
            return h * 3600 + m * 60 + s
        elif len(parts) == 2:
            m, s = map(int, parts)
            return m * 60 + s
    return np.nan

df['Czas_numeric'] = df['Czas'].apply(time_to_seconds)

# Przygotuj cechy: gender, age, pace_5k (w minutach na km!)
df['gender'] = df['Płeć'].map({'M': 0, 'K': 1})  # dostosuj mapowanie do swoich danych
current_year = pd.Timestamp.now().year
df['age'] = current_year - df['Rocznik']

# Użyj bezpośrednio kolumny '5 km Tempo', która już jest w minutach na km
df['pace_5k'] = df['5 km Tempo']

X = df[['gender', 'age', 'pace_5k']]
y = df['Czas_numeric']

# Usuń tylko wiersze z brakami w tych cechach
data = pd.concat([X, y], axis=1)
data = data.dropna(subset=['gender', 'age', 'pace_5k', 'Czas_numeric'])
X = data[['gender', 'age', 'pace_5k']]
y = data['Czas_numeric']

print('Liczba rekordów po czyszczeniu:', len(X))
print('Przykładowe wartości y (sekundy):', y.head())
print('Przykładowe wartości pace_5k (min/km):', X['pace_5k'].head())
print('Statystyki czasu:', y.describe())

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import Pipeline

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

models = {
    'LinearRegression': LinearRegression(),
    'RandomForest': RandomForestRegressor(random_state=42)
}
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    results[name] = {
        'MAE': mean_absolute_error(y_test, y_pred),
        'MSE': mean_squared_error(y_test, y_pred),
        'R2': r2_score(y_test, y_pred)
    }
print('Porównanie modeli:')
for name, metrics in results.items():
    print(f"{name}: {metrics}")

sorted_models = sorted(
    results.items(),
    key=lambda x: (-x[1]['R2'], x[1]['MAE'], x[1]['MSE'])
)
best_model_name = sorted_models[0][0]
best_model = models[best_model_name]
print(f"\nNajlepszy model (wg R2, MAE, MSE): {best_model_name}")

pipeline = Pipeline([
    ('model', best_model)
])
pipeline.fit(X_train, y_train)

import joblib
model_path = "best_model_pipeline.joblib"
joblib.dump(pipeline, model_path)
print(f"Pipeline zapisany do pliku: {model_path}")

import boto3
session = boto3.session.Session()
client = session.client('s3')
bucket = 'kursai'
client.upload_file(model_path, bucket, model_path)
print(f"Pipeline wysłany do Digital Ocean Spaces: {bucket}/{model_path}")

Liczba rekordów po czyszczeniu: 7009
Przykładowe wartości y (sekundy): 1    3983
3    4216
4    4227
6    4278
7    4302
Name: Czas_numeric, dtype: int64
Przykładowe wartości pace_5k (min/km): 1    2.960000
3    3.236667
4    3.240000
6    3.123333
7    3.300000
Name: pace_5k, dtype: float64
Statystyki czasu: count     7009.000000
mean      7197.278642
std       1220.741752
min       3864.000000
25%       6337.000000
50%       7081.000000
75%       7909.000000
max      12512.000000
Name: Czas_numeric, dtype: float64
Porównanie modeli:
LinearRegression: {'MAE': 279.6483647412662, 'MSE': 142174.92719967576, 'R2': 0.9085045267480649}
RandomForest: {'MAE': 309.4213135345198, 'MSE': 178743.20461004108, 'R2': 0.8849713207632263}

Najlepszy model (wg R2, MAE, MSE): LinearRegression
Pipeline zapisany do pliku: best_model_pipeline.joblib
Pipeline wysłany do Digital Ocean Spaces: kursai/best_model_pipeline.joblib


In [17]:
from sklearn.pipeline import Pipeline

# Stwórz pipeline z wybranymi cechami i najlepszym modelem
pipeline = Pipeline([
    ('selector', SelectKBest(score_func=f_regression, k=len(selected_features))),
    ('model', best_model)
])

# Dopasuj pipeline do całych danych treningowych (przekształć y na sekundy)
pipeline.fit(X_train, y_train.apply(time_to_seconds))

# Zapisz pipeline do pliku
model_path = "best_model_pipeline.joblib"
joblib.dump(pipeline, model_path)
print(f"Pipeline zapisany do pliku: {model_path}")

Pipeline zapisany do pliku: best_model_pipeline.joblib


In [18]:
session = boto3.session.Session()
client = session.client(
    's3',
)

bucket = 'kursai'  # zmień na swój bucket jeśli inny
client.upload_file(model_path, bucket, model_path)
print(f"Pipeline wysłany do Digital Ocean Spaces: {bucket}/{model_path}")

Pipeline wysłany do Digital Ocean Spaces: kursai/best_model_pipeline.joblib
